In [ ]:
import os
import json
from datetime import datetime
from dotenv import load_dotenv
from langchain_core.tools import tool
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, ToolMessage

# Load environment variables
load_dotenv()

# Initialize Azure OpenAI client via LangChain
model = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    #api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2023-07-01-preview",
    model=os.getenv("AZURE_OPENAI_MODEL_NAME"),
    temperature=0
)

### Find Country based on Location/Position

In [ ]:
import os
import json
from typing import TypedDict
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import END, START, StateGraph

# Initialize model
model_low_temp = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    #api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2023-07-01-preview",
    model=os.getenv("AZURE_OPENAI_MODEL_NAME"),
    temperature=0.1
)

# ---------------------------
# State Definitions
# ---------------------------
class State(TypedDict):
    user_input: str
    country_name: str
    aviation_regulator: str
    coordinates: str
    regulator_code: str

class Input(TypedDict):
    user_input: str

class Output(TypedDict):
    country_name: str
    aviation_regulator: str
    coordinates: str
    regulator_code: str
    original_instruction: str

# ---------------------------
# Prompt
# ---------------------------
system_prompt = SystemMessage(
    """You are an aviation assistant.  
    From the user input:
    1. Extract latitude and longitude if present.  
    2. Identify the country where the coordinates are located.  
    3. Provide the official aviation regulatory authority of that country.  
    4. Suggest a lowercase search code (without .pdf), e.g. faa, dgca, easa.  

    Respond ONLY in JSON:
    {
      "coordinates": "lat,lon" or "INVALID",
      "country_name": "Country" or "Unknown",
      "aviation_regulator": "Authority" or "Unknown",
      "regulator_code": "shortcode" or "unknown"
    }
    """
)

# ---------------------------
# Function
# ---------------------------
def process_input(state: State) -> State:
    user_message = HumanMessage(state["user_input"])
    res = model_low_temp.invoke([system_prompt, user_message])

    try:
        data = json.loads(res.content)
    except Exception:
        data = {
            "coordinates": "INVALID",
            "country_name": "Unknown",
            "aviation_regulator": "Unknown",
            "regulator_code": "unknown"
        }

    return {
        "coordinates": data.get("coordinates", "INVALID"),
        "country_name": data.get("country_name", "Unknown"),
        "aviation_regulator": data.get("aviation_regulator", "Unknown"),
        "regulator_code": data.get("regulator_code", "unknown"),
        "user_input": state["user_input"],
    }

# ---------------------------
# Build Graph
# ---------------------------
builder = StateGraph(State, input_schema=Input, output_schema=Output)
builder.add_node("process_input", process_input)
builder.add_edge(START, "process_input")
builder.add_edge("process_input", END)
graph = builder.compile()

# ---------------------------
# Interactive Usage
# ---------------------------
if __name__ == "__main__":
    while True:
        user_instruction = input("Enter your latitude and longitude : ").strip()
        if user_instruction.lower() in ["quit", "exit", "q"]:
            break

        result = graph.invoke({"user_input": user_instruction})

        print(f"Country and regulation : {result['country_name']} - {result['aviation_regulator']}")
        print(f"Search for file name (pdf type): {result['regulator_code']}")


Enter your latitude and longitude :  13.0843° N, 80.2705° E


Country and regulation : India - Directorate General of Civil Aviation
Search for file name : dgca


Enter your latitude and longitude :  exit


### LangChain RAG using Azure embedding

In [5]:
import os
from dotenv import load_dotenv

# from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import AzureOpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_openai import AzureChatOpenAI

load_dotenv()  # Ensure this is called first

# --- Azure model setup ---
model = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2023-07-01-preview",
    model=os.getenv("AZURE_OPENAI_MODEL_NAME"),  # e.g., "gpt-4o" or "gpt-35-turbo"
    temperature=0
)


# Single file load and process PDF
'''
pdf_path = r"C:\_IESEG\_Internship\_Lyneports\_LyneportProject\_regulations\EASA_USA\Copy of EASA Prototype Technical Design Specifications for Vertiports_March2022.pdf"
loader = PyPDFLoader(pdf_path)
'''

# Multiple file load from directory and process PDF
loader = DirectoryLoader(
    "./_regulations/FAA_USA",
    glob="**/*.pdf",  # recursive search
    loader_cls=PyPDFLoader
)

# Load all matching PDF files from the _regulations folder into a list of Document objects
documents = loader.load()

# Split into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

# Embeddings (Azure uses OpenAI API, so this works with correct API key)
embeddings = AzureOpenAIEmbeddings(
    model=os.getenv("AZURE_EMBEDDING_MODEL_NAME"),
    azure_endpoint=os.getenv("AZURE_EMBEDDING_ENDPOINT"),  # Correct key now
    openai_api_key=os.getenv("AZURE_EMBEDDING_API_KEY"),
    openai_api_version="2024-02-01",  # or whatever version you're using
)

# Create FAISS vector store
vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever()

# Create RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever
)

# Example query
# Summarize the design requirements for vertiport landing areas.
# What is VTOL-capable aircraft?
# What is MTOW?
# TLOF Guidance in 5 lines
# Takeoff and Landing Area Minimum Dimensions for TLOF

query = "Takeoff and Landing Area Minimum Dimensions for TLOF"

result = qa_chain.run(query)

print(result)


The minimum dimensions for the Takeoff and Landing Area (TLOF) are as follows:

- For non-powered-lift scenarios: 1.88 D-p
- For powered-lift scenarios: 1 D-p

These dimensions are specified in terms of length and width or diameter.


### Full and specific JSON output with Memory

In [6]:
### Full and specific JSON output with Memory
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# Initial system prompt
system_message = SystemMessage(
    content="""You are a civil aviation layout assistant that helps engineers, architects, and designers generate 3D-ready TLOF (Touchdown and Lift-Off Area) configurations from plain English descriptions, producing TLOF configurations in JSON format.
You understand and convert natural language into a structured JSON format for heliport design, including parameters like geometry, markings, lighting, safety areas, and landing markers.
You respect strict validation rules for shape types, colors, marker types, dimensions, and category activations.
Use default values where input is missing or unclear, and validate all parameters with explanations.

**Instructions for Handling Updates**:
- When the user requests an update to an existing TLOF configuration (e.g., 'update net height to 11'), respond with a minimal JSON object that includes only the updated field(s) within the same nested structure as the original (e.g., `{"TLOF": [{"dimensions": {"netHeight": 11}}]}`).
- Do not regenerate or include unchanged fields unless explicitly requested.
- Ensure the JSON structure remains compatible with the full TLOF configuration, preserving the `TLOF` array and `dimensions` object hierarchy.
- For new TLOF configurations, provide the full JSON structure with all required fields, using defaults for unspecified parameters.

**Validation Rules**:
- Validate shape types (e.g., 'Rectangle', 'Circle'), colors (e.g., 'white', 'blue'), marker types (e.g., 'V', 'H'), and dimensions.
- Use default values for missing parameters (e.g., `transparency`: 0, `baseHeight`: 0, `markingColor`: 'white').
- Ensure numerical values (e.g., `diameter`, `netHeight`) are positive and within realistic bounds.

**Example Input for New Configuration**:
Generate a rectangular TLOF for a tiltrotor aircraft with 30m x 40m dimensions, elevation 5m, rotation 15 degrees, and 0.6 transparency. Location is [139.6917, 35.6895]. Add a 'V' landing marker in blue, scaled to 8, rotated to 90 degrees.

**Example Output for New Configuration**:
{
  "TLOF": [
    {
      "position": [
        139.6917,
        35.6895
      ],
      "dimensions": {
        "unit": "m",
        "aircraft": "tiltrotor",
        "diameter": 1.0,
        "isVisible": true,
        "layerName": "Praveen_TLOF",
        "shapeType": "Rectangle",
        "scaleCategory": false,
        "textureScaleU": 1,
        "textureScaleV": 1,
        "safetyNetScaleU": 1,
        "safetyNetScaleV": 1,
        "aircraftCategory": false,
        "sides": 3,
        "width": 30,
        "length": 40,
        "height": 0.2,
        "rotation": 15,
        "transparency": 0.6,
        "baseHeight": 5,
        "markingsCategory": true,
        "markingType": "solid",
        "markingColor": "white",
        "markingThickness": 0.5,
        "dashDistance": 1,
        "dashLength": 1,
        "landingMarkerCategory": true,
        "landingMarker": "V",
        "markerScale": 8,
        "markerThickness": 0.02,
        "markerRotation": 90,
        "markerColor": "blue",
        "letterThickness": 0.5,
        "tdpcCategory": false,
        "tdpcType": "circle",
        "tdpcScale": 5,
        "tdpcThickness": 0.5,
        "tdpcRotation": 0,
        "tdpcExtrusion": 0.02,
        "tdpcColor": "white",
        "lightCategory": true,
        "lightColor": "white",
        "lightScale": 1,
        "lightDistance": 1,
        "lightRadius": 0.3,
        "lightHeight": 0.2,
        "safetyAreaCategory": false,
        "safetyAreaType": "multiplier",
        "dValue": 10,
        "multiplier": 1.5,
        "offsetDistance": 3,
        "safetyNetCategory": false,
        "curveAngle": 45,
        "netHeight": 15,
        "safetyNetTransparency": 0.5
      }
    }
  ]
}

**Example Input for Update**:
update net height to 11

**Example Output for Update**:
{
  "TLOF": [
    {
      "dimensions": {
        "netHeight": 11
      }
    }
  ]
}

Now generate the JSON for the user's input:"""
)

# Initialize message memory with the system message
messages = [system_message]

# Output parser
parser = StrOutputParser()

# Interactive assistant
print("🚁 Welcome! I'm your TLOF assistant. How can I help you? Type 'exit' to quit.\n")

while True:
    user_input = input("User: ").strip()
    if user_input.lower() in ["exit", "quit"]:
        break

    # Append user's message
    messages.append(HumanMessage(content=user_input))

    # Invoke the model with full history
    response = model.invoke(messages)

    # Add AI response to memory
    messages.append(AIMessage(content=response.content))

    # Output the assistant's response
    print("\nAssistant:\n", response.content, "\n")


🚁 Welcome! I'm your TLOF assistant. How can I help you? Type 'exit' to quit.



User:  create a landing surface for joby s4



Assistant:
 {
  "TLOF": [
    {
      "position": [
        0.0,
        0.0
      ],
      "dimensions": {
        "unit": "m",
        "aircraft": "joby s4",
        "diameter": 1.0,
        "isVisible": true,
        "layerName": "Default_TLOF",
        "shapeType": "Circle",
        "scaleCategory": false,
        "textureScaleU": 1,
        "textureScaleV": 1,
        "safetyNetScaleU": 1,
        "safetyNetScaleV": 1,
        "aircraftCategory": false,
        "sides": 3,
        "width": 20,
        "length": 20,
        "height": 0.2,
        "rotation": 0,
        "transparency": 0,
        "baseHeight": 0,
        "markingsCategory": true,
        "markingType": "solid",
        "markingColor": "white",
        "markingThickness": 0.5,
        "dashDistance": 1,
        "dashLength": 1,
        "landingMarkerCategory": false,
        "markerScale": 1,
        "markerThickness": 0.02,
        "markerRotation": 0,
        "markerColor": "white",
        "letterThickness": 0.5,

User:  update curve angle to 50



Assistant:
 {
  "TLOF": [
    {
      "dimensions": {
        "curveAngle": 50
      }
    }
  ]
} 



User:  exit
